# Import kaggle repository
*Movie Lens*

In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('grouplens/movielens-20m-dataset')

import zipfile
with zipfile.ZipFile("movielens-20m-dataset.zip","r") as zip_ref:
    zip_ref.extractall('')


In [10]:
import pandas as pd

from datetime import datetime 

import matplotlib.pyplot as plt

import numpy as np

tag=pd.read_csv("tag.csv")

genome_tags=pd.read_csv("genome_tags.csv")    

movie=pd.read_csv("movie.csv")

rating=pd.read_csv("rating.csv")

genome_scores=pd.read_csv("genome_scores.csv")